# SynBioPython & Opentrons (OT)

This notebook demonstrates how to export Synbiopython well transfers into an Opentrons API Python script.

---
## Section 0: script generator

In [1]:
import sys, string

class ScriptGen:
    """Class for generating a Python script."""

    def __init__(self, tab="    "):
        self.code = []
        self.tab = tab
        self.level = 0

    def script(self):
        return "".join(self.code)

    def write(self, string, end="\n"):
        self.code.append(self.tab * self.level + string + end)

    def indent(self):
        self.level = self.level + 1

    def unindent(self):
        if self.level == 0:
            raise SyntaxError
        self.level = self.level - 1

---
## Section 1: import OT plates

Open the Labware Library website (https://labware.opentrons.com) and obtain the `API NAME` of the desired plates. Here we use a 96-well plate (`biorad_96_wellplate_200ul_pcr`) and a 384-well plate (`corning_384_wellplate_112ul_flat`).

In [18]:
from synbiopython import lab_automation as lab
source = lab.Plate96()
destination = lab.Plate384()

In [19]:
source.name = 'biorad_96_wellplate_200ul_pcr'
destination.name = 'corning_384_wellplate_112ul_flat'

Alternatively, new classes can be created for these plates.

---
## Section 2: synbiopython plate & transfers

In [20]:
source.wells["A1"].add_content({"Compound_1": 1}, volume=5 * 10 ** (-6))
source.wells["C2"].add_content({"Compound_X": 1}, volume=15 * 10 ** (-6))

transfer = lab.Transfer(source.wells["C2"], destination.wells["G8"], 3 * 10 ** (-6))
transfer_2 = lab.Transfer(source.wells["A1"], destination.wells["I12"], 2 * 10 ** (-6))

---
## Section 3: OpentronsSetup

In [33]:
class OpentronsSetup:
    """Class for storing synbiopython transfers for exporting as an OT script.

    Parameters
    ----------
    
    plates
      dict in the format {Plate(): int}, where int is the plate's position on the OT machine (1--11)

    transfers
      dict: {'left': transfers1, 'right': transfers2}, 
      where transfers# is a list of Transfer objects or PickList().transfers_list for each pipette
    """

    def __init__(self, plates, transfers):
        self.plates = plates
        self.transfers = transfers
        self.metadata = {}

    def add_plate(self, plate, position):
        self.plates[plate] = position

    def export_script(self):
        """Export transfers as an OT API script."""

        def write_transfers(side, transfers):
            """Write transfers of one pipette, using ScriptGen.
            
            Parameters
            ----------

            side
              str key of OpentronsSetup().transfers

            transfers
              value of OpentronsSetup().transfers (list of Transfer objects)
            """

            for transfer in transfers:
                pipette = side + "_pipette"
                # we want opentrons volume (uL), from synbiopython volume (L=10**6 uL)
                volume = transfer.volume * 10**6

                source_plate = "plate_" + str(ot.plates[transfer.source_well.plate])
                source_well = transfer.source_well.name
                destination_plate = "plate_" + str(ot.plates[transfer.destination_well.plate])
                destination_well = transfer.destination_well.name

                c.write("%(pipette)s.transfer(%(volume)f, %(source_plate)s['%(source_well)s'], %(destination_plate)s['%(destination_well)s'])" % {
                    "pipette": pipette,
                    "volume": volume,
                    "source_plate": source_plate,
                    "source_well": source_well,
                    "destination_plate": destination_plate,
                    "destination_well": destination_well,
                    })


        c = ScriptGen()
        c.write("from opentrons import protocol_api")
        c.write("")
        c.write("# Parameters")
        c.write("left_pipette_name = ''")
        c.write("right_pipette_name = ''")

        c.write(("metadata = " + str(ot.metadata)))
        c.write("")

        c.write("def run(protocol: protocol_api.ProtocolContext):")
        c.indent()
        c.write("")
        c.write("# labware")
        for plate, pos in ot.plates.items():
            c.write("plate_%(pos)d = protocol.load_labware('%(name)s', %(pos)d)" % {"pos": pos, "name": plate.name})

        c.write("")
        c.write("# pipettes")
        c.write("left_pipette = protocol.load_instrument(left_pipette_name, 'left', tip_racks=[tiprack])")
        c.write("right_pipette = protocol.load_instrument(right_pipette_name, 'right', tip_racks=[tiprack])")

        c.write("")
        c.write("# commands")
        for side, transfers in ot.transfers.items():
            write_transfers(side, transfers)
            c.write("")

        # c.indent()
        # c.write("")
        # c.unindent()
        return(c.script())

---
## Section 4: generating an OT API script

In [34]:
plates = {source: 1, destination: 2}
transfers = {'left': [transfer, transfer_2], 'right': [transfer]}

In [35]:
ot = OpentronsSetup(plates=plates, transfers=transfers)
ot.metadata = {'apiLevel': '2.2'}

In [38]:
print(ot.export_script())

from opentrons import protocol_api

# Parameters
left_pipette_name = ''
right_pipette_name = ''
metadata = {'apiLevel': '2.2'}

def run(protocol: protocol_api.ProtocolContext):
    
    # labware
    plate_1 = protocol.load_labware('biorad_96_wellplate_200ul_pcr', 1)
    plate_2 = protocol.load_labware('corning_384_wellplate_112ul_flat', 2)
    
    # pipettes
    left_pipette = protocol.load_instrument(left_pipette_name, 'left', tip_racks=[tiprack])
    right_pipette = protocol.load_instrument(right_pipette_name, 'right', tip_racks=[tiprack])
    
    # commands
    left_pipette.transfer(3.000000, plate_1['C2'], plate_2['G8'])
    left_pipette.transfer(2.000000, plate_1['A1'], plate_2['I12'])
    
    right_pipette.transfer(3.000000, plate_1['C2'], plate_2['G8'])
    



In [39]:
# f = open('ot.py', 'wb')
# f.write(ot.export_script().encode('utf8'))
# f.close()